In [1]:
#### EMS Stroke Triage and Transport Model (ESSTM) code  ####
#### Contributor(s): Mehul Patel, Eliot McGinnis, Joseph Konstanzer, Matt Jansen ####
#### Last Modified: 4/11/2022 ####

### Cell 1 - imports packages ###

from scipy.stats import *
import math
import pandas
import csv
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import numpy as np


In [2]:
### Cell 2 - defines the simulation ###

from csv import writer

#Define the simulation - starting values provided, see modeling plan doc for model input parameters (Table 1)
def simulation(xCSC = .5, yCSC =.5, xPSC = .2, yPSC = .5, xPSC2 = .8, yPSC2 = .27, geoscale=10, sensitivity = .7, map_number=0,
                 specificity= .7, threshold = 20, patients = 1000, drivespeed = 35, runID = "A1", save = False, plot = True, 
             two_psc = True, scenario=0, seed=0):
    output_data = []
    
    xCSC *= geoscale
    yCSC *= geoscale
    xPSC *= geoscale
    yPSC *= geoscale
    xPSC2 *= geoscale
    yPSC2 *= geoscale
    threshold = threshold


    def diagnosePatient(hasLVO):
        if hasLVO:
            return bool(bernoulli.rvs(p = sensitivity))
        else:
            return (bool(bernoulli.rvs(p = (1 - specificity))))

    def runPatient1():         
        
        destination = ""
        correct_destination = ""
        timeInSystem = 0
        
        xLocation = uniform.rvs(0,1) * geoscale
        yLocation = uniform.rvs(0,1) * geoscale
        distFromCSC = math.hypot(xCSC - xLocation, yCSC - yLocation)
        distFromPSC = math.hypot(xPSC - xLocation, yPSC - yLocation)
        distFromPSC2 = math.hypot(xPSC2 - xLocation, yPSC2 - yLocation)
    ################################################ EMS transport time to hospital ###########################
        ############### P6 EMS transport speed ################
        if geoscale <= 60:
            drivespeed = 35 + (geoscale - 30)/2 #for region sizes 60x60 sq mi, drive speed increases with size
        else:
            drivespeed = 50                     #for region size >60x60 sq mi, drive speed set to 50 mph
        #
        time2PSC = (distFromPSC/drivespeed)*60 #dist divided by driving speed and then times 60 to get time in minutes
        time2CSC = (distFromCSC/drivespeed)*60
        time2PSC2 = (distFromPSC2/drivespeed)*60   
    

        timeAtScene = 0 # time spent by EMS on scene
        time2Scene = 0 # time for EMS to travel to scene
        time2Hospital = 0 # time from scene to hospital

        LVO_diagnosis = False
        result = "correct"
        closest_hospital = "PSC"
        confusion = "TN"
        
        if time2CSC < time2PSC:
            closest_hospital = "CSC"
        if two_psc:
            if time2PSC2 < time2PSC and time2PSC2 < time2CSC:
                closest_hospital = "PSC2"


        times = {"PSC" : time2PSC, "CSC" : time2CSC, "PSC2" : time2PSC2}
        ## TODO minimum from times

    ######################### P1 Stroke Diagnosis ############################            
        hasStroke = bool(bernoulli.rvs(p = .4))
        if hasStroke:
            if bool(bernoulli.rvs(p=.13)):
                hemorrhaging = True
                ischemic = False
            else:
                ischemic = True
                hemorrhaging = False

        else: 
            hemorrhaging = False
            ischemic = False
    ################################# P1 (cont'd) LVO #########################################
        if ischemic:
            if bool(bernoulli.rvs(p = .387)):
                hasLVO = True
                
            else:
                hasLVO = False
                
        else:
            hasLVO = False
            

        lastWell = uniform.rvs(0,1) * 100
        


############# P2 Time since LKW - random on the interval (a, a+b)
        if lastWell < 100:
            #last_well = uniform.rvs(4,6)
            #last_well = 5
            last_well = uniform.rvs(3,3)
           
        if lastWell <= 93.24:
            last_well = uniform.rvs(6,18)
            
        if lastWell <= 82.87:
            last_well = uniform.rvs(24,24)
            
        if lastWell <= 66.39:
            last_well = uniform.rvs(0,3)
            
        
############################### P3 + P4 EMS to scene time #######################################

        wait = norm.rvs(15.1, 7)

        time2Scene += 1.62 # In Bogle et. all, constant time between initial call and dispatch
        time2Scene += wait
        if time2Scene < 1:
            time2Scene = 1
            timeInSystem += 1

        else:    
            timeInSystem += 1.62
            timeInSystem += wait

    ######################### P5 EMS on scene time ########################################################
        wait = beta.rvs(2.91, 6.056, scale = 40)
        timeInSystem += wait
        timeAtScene += wait


    ########################## EMS TO HOSPITAL ###################################################################

        wait = 0


    ############################## True positive and negative ############################################           
        LVO_diagnosis = diagnosePatient(hasLVO)
        if LVO_diagnosis and hasLVO:
            confusion = "TP"
        elif LVO_diagnosis and not hasLVO: 
            confusion = "FP"
        elif not LVO_diagnosis and hasLVO:
            confusion = "FN"
            
        ###################### correct destination ###################3    
        correct_destination = closest_hospital    
        ##### override for patients with LVO diagnosis
        if hasLVO and last_well <= 24:
            correct_destination = "CSC"
    ############################## DESTINATION LOGIC ##########################################
        if not two_psc:
        ################# the one PSC case    
            if LVO_diagnosis and last_well <= 24:
                    
                if time2CSC - time2PSC < threshold:
                    destination = "CSC"
                    wait = time2CSC
                else:
                    destination = "PSC"
                    wait = time2PSC
            elif time2CSC < time2PSC:
                destination = "CSC"
                wait = time2CSC
            else:
                destination = "PSC"
                wait = time2PSC
        else:
    ####################### 2 PSC case ##################################
            if LVO_diagnosis and last_well <= 24:
                if times["CSC"] - times[closest_hospital] < threshold:
                    destination = "CSC"
                    wait = times["CSC"]
                    
                else:
                    destination = closest_hospital
                    wait = times[closest_hospital]
            else:

                destination = closest_hospital
                wait = times[closest_hospital]

        time2Hospital += wait
        timeInSystem += wait
        
############################# result logic #########################################################

        if (destination == "CSC") and (hasLVO == False) and (closest_hospital != "CSC"):
            #print("overtriage result")
            result = "overtriage"
        if hasLVO and destination != "CSC":
            result = "undertriage"

####################### within_threshold variable ################################3
        within_threshold = 0
        if times["CSC"] - times[closest_hospital] < threshold:
            within_threshold = 1

    ########################## RETURN PATIENT ############################################################# 
    
    
        return {"runID": runID, "map_number": map_number, "scenario": scenario, "sensitivity": sensitivity, "specificity": specificity,"transport threshold" : threshold, "hasStroke" : hasStroke, "hasLVO" : hasLVO, "LVO_diagnosis" : LVO_diagnosis, \
            "last_well": last_well, "destination" : destination, "xPSC" : xPSC, "yPSC" : yPSC, "xPSC2" : xPSC2, "yPSC2" : yPSC2, \
            "region_size" : geoscale, "correct_destination": correct_destination, "result": result, \
            "confusion": confusion, 'time2Scene': time2Scene, 'timeAtScene': timeAtScene, \
            "timeInSystem" : timeInSystem, "time2Hospital" : time2Hospital, \
            "closest_hospital": closest_hospital, "xLocation" : xLocation, "yLocation" : yLocation,
           "time2PSC": times["PSC"], "time2CSC": times["CSC"], 'time2PSC2': times["PSC2"], 'hemorrhaging': hemorrhaging,
           "ischemic": ischemic, "within_threshold": within_threshold, "distfromPSC": distFromPSC}
    
    

################### running the sim ###############################
    for i in range(0,patients):
        a = runPatient1()
        output_data.append(a)
    idx = pandas.Series(np.arange(1,patients+1), name = 'patient_id')
    rep_no = pandas.Series(np.ceil(idx/3000), name = 'rep_no')
    data = pandas.DataFrame(output_data)  
    data["seed"]=seed
    data = pandas.concat([idx, rep_no,data], axis = 1)
    print(data)
    if save:
        if not os.path.isfile("run.csv"):
            data.to_csv('run.csv', mode = 'w', header = True)
        else:
            data.to_csv('run.csv', mode= 'a', header=False)

        
    return output_data


In [5]:
%%capture

### Cell #3 runs the simulation - SET LOCAL DIRECTORY BELOW ###


from scipy.stats import *
import math
import csv
import os

# SET DIRECTORY TO SAVE CSV OUTPUT FILE 
os.chdir(r"C:\tempdat\strokesim")

### generate PSC 1 for run A, x and y coordinate
def randomMap():
    # region size varies from 20x20 sq mi to 100x100 sq mi
    geoscale=uniform.rvs(20,80)
    PSC_A1 = [uniform.rvs(0,1), uniform.rvs(0,1)]
    while math.hypot(.5 - PSC_A1[0], .5 - PSC_A1[1]) * geoscale < 1:
        print("PSC1 within one mile of CSC")
        PSC_A1 = [uniform.rvs(0,1), uniform.rvs(0,1)]
        
    psc_A1 = [i * geoscale for i in PSC_A1] 
    
    PSC_A2 = [uniform.rvs(0,1), uniform.rvs(0,1)]
    while math.hypot(.5 - PSC_A2[0], .5 - PSC_A2[1]) * geoscale < 1 or \
        math.hypot(PSC_A1[0] - PSC_A2[0], PSC_A1[1] - PSC_A2[1]) * geoscale < 1:
        
        print("PSC2 within one mile of hospital")
        print(PSC_A2)
        print(PSC_A1)
        PSC_A2 = [uniform.rvs(0,1), uniform.rvs(0,1)]


    return(PSC_A1, PSC_A2, geoscale)

if os.path.isfile("run.csv"):
    os.remove("run.csv")
else:
    print("cannot remove")

np.random.seed(421)
#SET i below for number of maps - random geoscale and placement of PSCs
for i in range(0,40):
    # for each random map
    np.random.seed(421+i)
    PSC1,PSC2, geoscale = randomMap()
    threshold=0
    sensitivity=.7
    specificity=.7
    scenario=0
    map_number=i
    ID = "_%d_%0.1f_%0.1f_%d" % (i,sensitivity, specificity, threshold)

    #SET scenarios based on sensitivity, specificity, and threshold
    for j in range(0,3):
        # loop for each threshold
        if j==0:
            sensitivity = .8
            specificity = .8
        elif j==1:
            sensitivity = .9
            specificity = .7
        else:
            sensitivity = .7
            specificity = .9
            #change below for threshold
        for l in range(0,7):
            threshold= l*10
            #rep_no=i
            ID = "_%d_%0.1f_%0.1f_%d" % (i,sensitivity, specificity, threshold)
            scenario=scenario+1
            np.random.seed(421) #keeping the random seed is keeping the distribution from varying
            #SET m below for random number seed replication
            for m in range(0,30):
                ### NEW STUFF  ###
                seed = m
                #np.random.seed(421) #keeping the random seed is keeping the distribution from varying
                sim = simulation(xPSC = PSC1[0], yPSC = PSC1[1], xPSC2 = PSC2[0], yPSC2 = PSC2[1], map_number=map_number,
                            sensitivity=sensitivity, specificity=specificity, geoscale=geoscale, scenario=scenario,
                            patients = 200, save = True, runID = ID, threshold=threshold, seed=seed);
                         

 
  # Number of maps times scenarios times random number seeds times patients will produce X data rows in run.csv
 
